In [1]:
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import math
import datetime

%matplotlib inline
%run function.ipynb

In [2]:
data = pd.read_csv('model1.csv')
data = data.dropna()
data = data.drop(['DisbursePerMonth', 'DisbursePerEmployee'], axis=1)
data

,State,Term,NoEmp,NewExist,IsFranchised,UrbanRural,RevLineCr,LowDoc,DisbursementGross,Default,Industry,Bank%,State_division,Is_Cross_State,Appv_Dusburse_delta,RealEstate,Recession,Mon_Is_14710,rate
0,NC,162.0,2.0,0.0,1.0,1,0.0,0.0,3.071757e+05,0.0,Retail_trade,25.0,South East,1,52.0,0.0,0.0,0.0,4.47
1,AR,126.0,7.0,1.0,0.0,1,0.0,0.0,1.664374e+05,0.0,Accom/Food_serv,15.0,South East,1,82.0,0.0,0.0,1.0,4.47
2,CA,83.0,18.0,0.0,0.0,1,1.0,0.0,5.316067e+05,0.0,Educational,50.0,West,1,21.0,0.0,0.0,0.0,4.47
3,FL,84.0,4.0,1.0,0.0,1,1.0,0.0,6.235643e+04,0.0,Construction,50.0,South East,0,21.0,0.0,0.0,0.0,4.47
4,LA,60.0,3.0,1.0,0.0,1,0.0,0.0,6.061084e+04,0.0,RE/Rental/Lease,15.0,South East,0,113.0,0.0,0.0,0.0,4.47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
518436,CO,84.0,2.0,1.0,0.0,0,1.0,0.0,3.636650e+04,0.0,Educational,50.0,South West,1,22.0,0.0,0.0,0.0,4.51
518437,AZ,29.0,2.0,0.0,0.0,1,-1.0,0.0,2.060768e+05,1.0,Other_no_pub,25.0,South West,1,22.0,0.0,0.0,0.0,4.51
518438,NV,240.0,20.0,1.0,0.0,1,-1.0,0.0,6.170183e+05,0.0,Construction,0.0,West,0,464.0,1.0,0.0,0.0,4.51
518439,AZ,84.0,3.0,1.0,0.0,1,1.0,0.0,1.895301e+04,0.0,Information,50.0,South West,1,22.0,0.0,0.0,0.0,4.51


In [3]:
y = data['Default']
X = data.drop('Default', axis=1)

In [4]:
X.columns

Index(['State', 'Term', 'NoEmp', 'NewExist', 'IsFranchised', 'UrbanRural',
       'RevLineCr', 'LowDoc', 'DisbursementGross', 'Industry', 'Bank%',
       'State_division', 'Is_Cross_State', 'Appv_Dusburse_delta', 'RealEstate',
       'Recession', 'Mon_Is_14710', 'rate'],
      dtype='object')

In [5]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score

In [6]:
X_dum = pd.get_dummies(X, columns=['State', 'Industry', 'State_division'], dtype=np.float64)

In [7]:
data.dtypes

State                   object
Term                   float64
NoEmp                  float64
NewExist               float64
IsFranchised           float64
UrbanRural               int64
RevLineCr              float64
LowDoc                 float64
DisbursementGross      float64
Default                float64
Industry                object
Bank%                  float64
State_division          object
Is_Cross_State           int64
Appv_Dusburse_delta    float64
RealEstate             float64
Recession              float64
Mon_Is_14710           float64
rate                   float64
dtype: object

In [8]:
X_dum

,Term,NoEmp,NewExist,IsFranchised,UrbanRural,RevLineCr,LowDoc,DisbursementGross,Bank%,Is_Cross_State,...,Industry_Unknown,Industry_Utilities,Industry_Wholesale_trade,State_division_Mid-Atlantic,State_division_Mid-West,State_division_North East,State_division_North West,State_division_South East,State_division_South West,State_division_West
0,162.0,2.0,0.0,1.0,1,0.0,0.0,3.071757e+05,25.0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,126.0,7.0,1.0,0.0,1,0.0,0.0,1.664374e+05,15.0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,83.0,18.0,0.0,0.0,1,1.0,0.0,5.316067e+05,50.0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,84.0,4.0,1.0,0.0,1,1.0,0.0,6.235643e+04,50.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,60.0,3.0,1.0,0.0,1,0.0,0.0,6.061084e+04,15.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
518436,84.0,2.0,1.0,0.0,0,1.0,0.0,3.636650e+04,50.0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
518437,29.0,2.0,0.0,0.0,1,-1.0,0.0,2.060768e+05,25.0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
518438,240.0,20.0,1.0,0.0,1,-1.0,0.0,6.170183e+05,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
518439,84.0,3.0,1.0,0.0,1,1.0,0.0,1.895301e+04,50.0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [9]:
X_dum.iloc[[142, 320, 414], :20]

,Term,NoEmp,NewExist,IsFranchised,UrbanRural,RevLineCr,LowDoc,DisbursementGross,Bank%,Is_Cross_State,Appv_Dusburse_delta,RealEstate,Recession,Mon_Is_14710,rate,State_AK,State_AL,State_AR,State_AZ,State_CA
142,0.0,10.0,1.0,0.0,1,1.0,0.0,50727.633458,50.0,1,52.0,0.0,0.0,0.0,4.47,0.0,0.0,0.0,0.0,0.0
320,0.0,1.0,0.0,0.0,1,0.0,0.0,105288.959600,50.0,1,47.0,0.0,0.0,0.0,0.98,0.0,0.0,0.0,0.0,0.0
414,0.0,1.0,1.0,0.0,1,1.0,0.0,47802.503770,50.0,1,44.0,0.0,0.0,1.0,1.01,0.0,0.0,0.0,0.0,0.0


In [10]:
X_dum.iloc[[142, 320, 414], 20:]

,State_CO,State_CT,State_DC,State_DE,State_FL,State_GA,State_HI,State_IA,State_ID,State_IL,...,Industry_Unknown,Industry_Utilities,Industry_Wholesale_trade,State_division_Mid-Atlantic,State_division_Mid-West,State_division_North East,State_division_North West,State_division_South East,State_division_South West,State_division_West
142,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
320,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
414,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [11]:
X_train, X_test, y_train, y_test = train_test_split(X_dum, y, random_state=0, test_size=0.25)

In [12]:
y_train

328473    1.0
205772    0.0
245746    0.0
17175     1.0
349575    0.0
         ... 
439107    0.0
117952    0.0
435829    0.0
305711    0.0
461484    0.0
Name: Default, Length: 388830, dtype: float64

In [13]:
np.all(np.isfinite(y_train))

True

In [14]:
y_train.describe()

count    388830.000000
mean          0.257601
std           0.437314
min           0.000000
25%           0.000000
50%           0.000000
75%           1.000000
max           1.000000
Name: Default, dtype: float64

In [15]:
forest = RandomForestClassifier()

print('start:', datetime.datetime.now())
forest.fit(X_train, y_train)
y_pred = forest.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print('CV:', np.mean(cross_val_score(forest, X_train, y_train.values.ravel())))
print("Accuracy: ", acc)
print('end:', datetime.datetime.now())

start: 2021-09-17 01:36:56.888342
CV: 0.9295810508448422
Accuracy:  0.9289412164091011
end: 2021-09-17 01:44:13.248391


In [16]:
from sklearn import metrics
confusion_matrix = metrics.confusion_matrix(y_test, y_pred, labels=[1, 0])
print('Confusion Matrix :')
print(confusion_matrix)
print('recall_score:', metrics.recall_score(y_test, y_pred, zero_division=0))
print('f_score:', metrics.f1_score(y_test, y_pred, zero_division=0))
print('precision_score:', metrics.precision_score(y_test, y_pred, zero_division=0))

Confusion Matrix :
[[26971  6584]
 [ 2626 93430]]
recall_score: 0.8037848308746833
f_score: 0.8541613883962502
precision_score: 0.9112747913639896


# RF tune parameter

In [17]:
from sklearn.model_selection import KFold, GridSearchCV, RandomizedSearchCV

In [18]:
print('start:', datetime.datetime.now())
forest_t = RandomForestClassifier()

# 設定要訓練的超參數組合
n_estimators = [500, 700, 900, 1200]
max_depth = [22, 26, 30, 35]
param_grid = dict(n_estimators=n_estimators, max_depth=max_depth)

## 建立搜尋物件，放入模型及參數組合字典 (n_jobs=-1 會使用全部 cpu 平行運算)
grid_search = GridSearchCV(forest_t, param_grid, scoring='accuracy', n_jobs=-1, verbose=1)

# 開始搜尋最佳參數
grid_result = grid_search.fit(X_train, y_train)

# 印出最佳結果與最佳參數
print("Best Accuracy: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

print(grid_result.best_params_)
print('end:', datetime.datetime.now())

start: 2021-09-17 01:44:13.945526
Fitting 5 folds for each of 16 candidates, totalling 80 fits


D:\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [0.90883677 0.90950801 0.90952344 0.90920454 0.91933236 0.91922177
 0.91923206 0.91881799 0.92457886 0.92458401 0.92444256 0.92457115
 0.92882236 0.92864748        nan        nan]
  warnings.warn(


Best Accuracy: 0.928822 using {'max_depth': 30, 'n_estimators': 500}
{'max_depth': 30, 'n_estimators': 500}
end: 2021-09-17 04:08:12.207150


In [19]:
forest_bestparam = RandomForestClassifier(max_depth=grid_result.best_params_['max_depth'], 
                                          n_estimators=grid_result.best_params_['n_estimators'])

# 訓練模型
forest_bestparam.fit(X_train, y_train)

# 預測測試集
y_pred = forest_bestparam.predict(X_test)
print('CV:', np.mean(cross_val_score(forest_bestparam, X_train, y_train.values.ravel())))
print("Accuracy: ", acc)

print('Confusion Matrix :')
print(confusion_matrix)
print('recall_score:', metrics.recall_score(y_test, y_pred, zero_division=0))
print('f_score:', metrics.f1_score(y_test, y_pred, zero_division=0))
print('precision_score:', metrics.precision_score(y_test, y_pred, zero_division=0))

CV: 0.9286140472700151
Accuracy:  0.9289412164091011
Confusion Matrix :
[[26971  6584]
 [ 2626 93430]]
recall_score: 0.8034570108776635
f_score: 0.8532050572020823
precision_score: 0.9095202752850685
